# DomoJupyter (GetInstanceCredentials)

> a function for interacting with DomoJupyter Accounts and Credentials

In [ ]:
# | default_exp integrations.DomoJupyter

In [ ]:
#| exporti
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum


import re
import pandas as pd
import importlib
import datetime as dt
import time

import json

# import domojupyter as dj

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.Logger as lc
import domolibrary.classes.DomoDataset as dmds

from fastcore.basics import patch_to


In [ ]:
# | export

class NoConfigCompanyError(Exception):
    def __init__(self, sql, domo_instance):
        message = f'SQL "{sql}" returned no results in {domo_instance}'
        self.message = message
        super().__init__(self.message)


class GetInstanceConfig:
    config: pd.DataFrame = None
    logger: lc.Logger = None

    def __init__(self, logger: Optional[lc.Logger] = None):

        self.logger = logger or lc.Logger(app_name='GetInstanceConfig')

    async def _retrieve_company_ds(self,
                                   config_auth: dmda.DomoAuth,
                                   dataset_id: str,
                                   sql: str,
                                   debug_prn: bool = False,
                                   debug_api: bool = False,
                                   debug_log: bool = False,
                                   ) -> pd.DataFrame:

        ds = await dmds.DomoDataset.get_from_id(auth=config_auth,
                                                dataset_id=dataset_id, debug_api=debug_api)

        message = f"⚙️ START - Retrieving company list \n{ds.display_url()} using \n{sql}"

        if debug_prn:
            print(message)

        logger.log_info(message, debug_log=debug_log)

        df = await ds.query_dataset_private(auth=config_auth,
                                            dataset_id=dataset_id,
                                            sql=sql,
                                            debug_api=debug_api)
        if len(df.index) == 0:
            raise NoConfigCompanyError(
                sql, domo_instance=config_auth.domo_instance)

        self.config = df

        message = f"\n⚙️ SUCCESS 🎉 Retrieved company list \nThere are {len(df.index)} companies to update"

        if debug_prn:
            print(message)
        logger.log_info(message, debug_log=debug_log)

        return df


In [ ]:
#| hide
import os

config_auth = dmda.DomoTokenAuth(domo_instance = 'domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

logger = lc.Logger( app_name='test_retrieve_company')

gic = GetInstanceConfig(logger = logger)

await gic._retrieve_company_ds(config_auth = config_auth,
                               dataset_id = '8d2a8055-7918-4039-b67d-361647e96ea8',
                               sql = 'SELECT domain from Table',
                               debug_prn = True,
                               debug_log = False,
                               debug_api = False
                            )

pd.DataFrame(gic.config)
pd.DataFrame(logger.logs)

⚙️ START - Retrieving company list 
https://domo-dojo.domo.com/datasources/8d2a8055-7918-4039-b67d-361647e96ea8/details/overview using 
SELECT domain from Table

⚙️ SUCCESS 🎉 Retrieved company list 
There are 1 companies to update


,date_time,application,log_type,log_message,breadcrumb,domo_instance,entity_id,function_name,file_name,function_trail
0,2023-01-18 00:06:13.650285,test_retrieve_company,Info,⚙️ START - Retrieving company list \nhttps://d...,,None,None,_retrieve_company_ds,/tmp/ipykernel_12923/1008752541.py,<module> -> _retrieve_company_ds
1,2023-01-18 00:06:14.173217,test_retrieve_company,Info,\n⚙️ SUCCESS 🎉 Retrieved company list \nThere ...,,None,None,_retrieve_company_ds,/tmp/ipykernel_12923/1008752541.py,<module> -> _retrieve_company_ds


## Get Domains with Global Config (DEPRECATED)
Use this method to configure a dataset that retrieves a list of domains from a config instance (using config credentials) and then includes a global_auth or global_exception_auth for each retrieved domo_instance which will be used to configure the instance.

Theoretically, the global user should be an Admin alreado Domo'ed to the instance

The Config Dataset must return columns **domo_instance** and **config_exception_pw**

NOTE: this method works as designed, but `get_domains_with_instance_auth` method is more flexible as it supports more variations by using an `Enum` class for matching.

In [ ]:
#| export
class GetDomains_Query_Exception_PW_Col_Error(Exception):
    """raise if SQL query fails to return column named 'config_exception_pw'"""

    def __init__(self, sql :str = None, domo_instance : str = None, message : str = None):
        message = message or f"Query failed to return a column 'config_exception_pw' sql = {sql} in {domo_instance}"
        super().__init__(self, message)


@patch_to(GetInstanceConfig, cls_method=True)
async def get_domains_with_global_config_auth(cls: GetInstanceConfig,

                                              config_dataset_id: str,
                                              config_auth: dmda.DomoAuth, # which instance to retrieve configuration data from
                                              
                                              global_auth: dmda.DomoAuth, # global authentication credentials
                                              global_exception_auth: dmda.DomoAuth, # exception credentials (ex 24 char password)
                                              # must return a column named domo_instance, if there is an exception_auth, must return a column 'config_exception_pw'
                                              
                                              config_sql: str = "select domain as domo_instance, config_exception_pw from table",
                                              
                                              debug_api: bool = False,
                                              debug_log: bool = False,
                                              debug_prn: bool = False,
                                              
                                              logger: lc.Logger = None) -> pd.DataFrame:
    if not logger:
        logger = lc.Logger(app_name='get_domains_with_global_config_auth')

    gic = cls(logger = logger)

    df = await gic._retrieve_company_ds(config_auth=config_auth,
                                        dataset_id=config_dataset_id,
                                        sql=config_sql,
                                       debug_prn=debug_prn,
                                       debug_log=debug_log,
                                       debug_api=debug_api)
    
    if 'config_exception_pw' not in df.columns:
        message = f"Query failed to return a column 'config_exception_pw' sql = {config_sql} in {config_auth.domo_instance}"
        gic.logger.log_error(message)
        raise GetDomains_Query_Exception_PW_Col_Error(message= message)

    for index, instance in df.iterrows():
        creds = global_auth

        if instance['config_exception_pw'] == 1:
            creds = global_exception_auth

        creds.domo_instance=instance['domo_instance']

        try:
            await creds.get_auth_token()
            df.at[index, 'is_valid'] = 1

        except dmda.InvalidCredentialsError as e:
            if debug_prn:
                print(e)
            
            logger.log_error(str(e))
            df.at[index, 'is_valid'] = 0
        
        finally:
            df.at[index, 'instance_auth'] = creds

    return df


#### sample implementation of `get_domains_with_global_config_auth`

In [ ]:
import os

config_auth = dmda.DomoTokenAuth(
    domo_instance='domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

global_auth = dmda.DomoTokenAuth(
    domo_instance='domo-global', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

global_exception_auth = dmda.DomoTokenAuth(
    domo_instance='domo-global', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])


logger = lc.Logger(app_name='test_retrieve_company')

res = await GetInstanceConfig.get_domains_with_global_config_auth(config_auth=config_auth,
                                                            config_dataset_id='8d2a8055-7918-4039-b67d-361647e96ea8',
                                                                  config_sql='SELECT domain as domo_instance, 1 as config_exception_pw  from Table',
                               debug_prn=True,
                               debug_log=False,
                               debug_api=False,
                               logger=logger,
                               global_auth = global_auth,
                                                                  global_exception_auth=global_exception_auth
                               )

pd.DataFrame(res)
# pd.DataFrame(logger.logs)

⚙️ START - Retrieving company list 
https://domo-dojo.domo.com/datasources/8d2a8055-7918-4039-b67d-361647e96ea8/details/overview using 
SELECT domain as domo_instance, 1 as config_exception_pw  from Table

⚙️ SUCCESS 🎉 Retrieved company list 
There are 1 companies to update


,domo_instance,config_exception_pw,is_valid,instance_auth
0,domo-dojo,1,1.0,"DomoTokenAuth(domo_instance='domo-dojo', token..."


## Get Domains with Instance Config

Use this method to configure a dataset that retrieves a list of domains from a config instance (using config credentials).  Pass an `auth_enum` object to enumerate different authenticaiton variations to expect in the result dataset (see example).

Theoretically, each of the enumerated auth variations should already exist in the instance. 

The Config Dataset must return columns **domo_instance** and **auth_match_col**

In [ ]:
# | export

class GetDomains_Query_AuthMatch_Error(Exception):
    """raise if SQL query fails to return column named 'auth_match_col'"""
    def __init__(self, sql : str = None, domo_instance : str = None ,message : str = None):
        message = message or f"Query failed to return a column 'auth_match_col' sql = {sql} in {domo_instance}"
        super().__init__(self, self.message)

@patch_to(GetInstanceConfig, cls_method=True)
async def get_domains_with_instance_auth(cls: GetInstanceConfig,
                                         config_dataset_id: str,
                                         
                                         config_auth: dmda.DomoAuth,  # which instance to retrieve configuration data from
                                         default_auth : dmda.DomoAuth,
                                         auth_enum : Enum,
                                         
                                         config_sql: str = "select domain as domo_instance,concat(config_useprod, '-', project) as auth_match_col from table",
                                         
                                        
                                         debug_api: bool = False,
                                         debug_log: bool = False,
                                         debug_prn: bool = False,
                                         logger: Optional[lc.Logger] = None) -> pd.DataFrame: # returns a dataframe with domo_instance, instance_auth, and binary column is_valid 
    if not logger:
        logger = lc.Logger(app_name='get_domains_with_instance_auth')

    gic = cls(logger=logger)

    df = await gic._retrieve_company_ds(config_auth=config_auth,
                                        dataset_id=config_dataset_id,
                                        sql=config_sql,
                                        debug_prn=debug_prn,
                                        debug_log=debug_log,
                                        debug_api=debug_api)

    if 'auth_match_col' not in df.columns:
        message = f"Query failed to return a column 'auth_match_col' sql = {config_sql} in {config_auth.domo_instance}"
        raise GetDomains_Query_AuthMatch_Error(message)

    for index, instance in df.iterrows():
        
        creds = default_auth

        match_auth = next(
            (member.value for member in auth_enum if member.name == instance['auth_match_col']))

        if match_auth:
            creds = match_auth

        creds.domo_instance=instance['domo_instance']

        try:
            await creds.get_auth_token(debug_api = debug_api)
            df.at[index, 'is_valid'] = 1

        except dmda.InvalidCredentialsError as e:
            if debug_prn:
                print(e)
            
            logger.log_error(str(e))
            df.at[index, 'is_valid'] = 0
        
        finally:
            df.at[index, 'instance_auth'] = creds

    return df


#### sample implementation of `get_domains_with_instance_auth`

In [ ]:
import os

config_auth = dmda.DomoTokenAuth(
    domo_instance='domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

default_auth = dmda.DomoTokenAuth(
    domo_instance='default', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])


class AuthEnum(Enum):
    test_1 = dmda.DomoTokenAuth(
        domo_instance='test_1', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])
    test_0 = dmda.DomoTokenAuth(
        domo_instance='test_0', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])


logger = lc.Logger(app_name='test_retrieve_company')

res = await GetInstanceConfig.get_domains_with_instance_auth(config_auth=config_auth,
                                                             config_dataset_id='8d2a8055-7918-4039-b67d-361647e96ea8',
                                                             config_sql="SELECT domain as domo_instance, 'test_1' as auth_match_col from Table",
                                                             debug_prn=True,
                                                             debug_log=False,
                                                             debug_api=False,
                                                             logger=logger,
                                                             default_auth = default_auth,
                                                             auth_enum=AuthEnum
                                                             )

pd.DataFrame(res)
# pd.DataFrame(logger.logs)


⚙️ START - Retrieving company list 
https://domo-dojo.domo.com/datasources/8d2a8055-7918-4039-b67d-361647e96ea8/details/overview using 
SELECT domain as domo_instance, 'test_1' as auth_match_col from Table

⚙️ SUCCESS 🎉 Retrieved company list 
There are 1 companies to update


,domo_instance,auth_match_col,is_valid,instance_auth
0,domo-dojo,test_1,1.0,"DomoTokenAuth(domo_instance='domo-dojo', token..."
